# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

![payload j1939](./images/j1939-payload.png)

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

## Imports

In [ ]:
from sqlmodel import Session, select

from canlock.db.database import get_session, init_db
from canlock.db.models import Vehicle, CanMessage

In [ ]:
init_db()  # Ensure tables are created

with get_session() as session:
    # Fetch all vehicles
    vehicles = session.exec(select(Vehicle)).all()
    
    # Fetch 100 CAN messages for example
    canmessages = session.exec(select(CanMessage).limit(100)).all()

In [ ]:
vehicles

In [ ]:
canmessages